## **Template de RAG Experimental**

> *Pipeline modular de Recuperação e Geração com Langchain + OpenAI*

**`Data:`** **Julho de 2025**

**`Objetivo:`** Este notebook tem como objetivo servir como **template base para construção de pipelines RAG (Retrieval-Augmented Generation)** usando Langchain. 

**`Autoria:`** Desenvolvido pelo  **Núcleo de Inteligência Artificial do PD&I** da Radix. 

**`Equipe Técnica:`**

|<img src="https://i.imgur.com/HvkmiXf.png" width="100"> | <img src="https://i.imgur.com/27pmxnN.jpeg" width="100" > | <img src="https://i.imgur.com/enG4rPa.jpeg" width="100"> | <img src="https://i.imgur.com/FSL0u7D.jpeg" width="100" > |
|:--:|:--:|:--:|:--:|
| Maria Júlia Vidal | Éric Kauati | Milena Toledo | Fabio Contrera |



## **RAG (Retrieval-Augmented Generation)** 
##### **RAG** é sigla para *Retrieval Augmented Generation*, uma técnica que amplia a capacidade de resposta de modelos de linguagem (LLMs) ao combinar seu conhecimento interno com informações recuperadas de fontes externas.


O funcionamento da RAG pode ser entendido como uma colaboração entre dois componentes:

- **Recuperação (Retriever):** busca informações relevantes em documentos externos (como PDFs, textos ou bancos de dados vetoriais).
- **Geração (Generator):** o modelo de linguagem usa essas informações, junto com seu próprio conhecimento, para produzir uma resposta precisa e contextualizada.


### Como o processo funciona na prática:

1. A pessoa usuária faz uma pergunta inicial.
2. O sistema identifica quais informações são relevantes em fontes externas. 
3. As informações relevantes são passadas como contexto. 
4. O contexto é juntado com a entrada inicial do usuário e passada para a LLM.
5. O modelo de linguagem analisa tudo e gera uma resposta integrada.

    <img src="https://i.imgur.com/dyVzasZ.png" width="1000">




### Conceitos fundamentais:

- **Langchain:** framework especializado em coordenar o uso de LLMs, simplificando o desenvolvimento de aplicações complexas com modelos de linguagem, como a RAG.
- **Chunking:** divisão de textos longos em pedaços menores (*chunks*), o que torna a busca mais eficiente e garante que os textos estejam no tamanho limite de contexto do modelo.
- **Embeddings:** representações vetoriais dos textos, usadas para medir a similaridade entre uma pergunta e os documentos. Os embeddings são uma forma de buscar por significado. 
- **Vector Store:** banco vetorial onde são armazenados os embeddings dos documentos. É ele que permite a recuperação rápida e eficiente dos chunks mais relevantes durante uma consulta.
- **LCEL:** é sigla para Langchain Expression Language, que é uma forma modular, declarativa e encadeável de integrar todas as etapas do pipeline no Langchain.


`Fontes:`
- O que é RAG?: https://www.alura.com.br/artigos/o-que-e-rag#:~:text=RAG%20%28Retrieval%20Augmented%20Generation%29%20%C3%A9%20uma%20t%C3%A9cnica%20utilizada,de%20linguagem%20com%20sistemas%20de%20recupera%C3%A7%C3%A3o%20de%20informa%C3%A7%C3%B5es.

- RAG: https://medium.com/blog-do-zouza/rag-retrieval-augmented-generation-8238a20e381d



> A seguir, vamos explorar cada etapa necessária para construir a nossa pipeline de RAG usando o Langchain. Antes, **leia o README.md** para realizar o *setup*  do ambiente e entender a configuração básica desse *template*.  


### Instalações:

Antes de tudo, é necessário instalar as bibliotecas que serão usadas. Como explicado no README.md, faremos isso pelo arquivo requirements.txt.

In [ ]:

%pip install -r requirements.txt
%pip install ipykernel

---



## **`1`** - **Importações**

Depois de instalar as bibliotecas, vamos importar os módulos necessários.

In [19]:

import os # biblioteca padrão do Python para interagir com o sistema operacional (acessar variáveis do ambiente, trabalhar com chaves de API, ou com caminhos de arquivos). 
from langchain_openai import OpenAIEmbeddings # gerador de embeddings da OpenAI, convertendo texto em vetores numéricos.
from langchain_community.document_loaders import PyPDFLoader # carregar conteúdo de arquivos PDF, convertendo para um formato legível pelo LangChain.
from langchain.text_splitter import RecursiveCharacterTextSplitter # ferramenta de chunking, divide o texto em partes menores com sobreposições.
from langchain_core.documents import Document # define a estrutura básica de um documento em Langchain, que contém texto e metadados.
from langchain_core.vectorstores import InMemoryVectorStore # implementação de armazenamento vetorial em memória, permitindo a busca por similaridade.
from langchain_core.prompts import ChatPromptTemplate # usado para criar prompts personalizados, combinando variáveis, contexto e perguntas dentro de um único template estruturado. 
from langchain_core.output_parsers import StrOutputParser # converter a resposta gerada pelo modelo em uma string limpa
from langchain_core.runnables import RunnablePassthrough # componente auxiliar usado em LCEL para passar os dados adiante sem alterações, conectando partes do pipeline que ainda não precisam ser processadas. 
from langchain.chains import create_retrieval_chain # cria a cadeia de recuperação da RAG, desde a entrada do usuário até o output final.
from langchain.chains.combine_documents import create_stuff_documents_chain # cria uma cadeia de geração (stuffing) onde os documentos recuperados são empacotados no prompt para gerar a resposta.

from dotenv import load_dotenv # biblioteca para carregar variáveis de ambiente de um arquivo .env, facilitando a configuração de chaves de API e outras configurações sensíveis.
load_dotenv() # chama a função load_dotenv para carregar as variáveis de ambiente do arquivo .env.

# Recupera a variável de ambiente OPENAI_API_KEY e armazena esse valor na variável API_KEY_OPENAI.
API_KEY_OPENAI = os.getenv("OPENAI_API_KEY")
# Coloca a chave da API manualmente no ambiente do sistema operacional, garantindo que tenha o valor certo e que funcione. 
os.environ["OPENAI_API_KEY"] = API_KEY_OPENAI

---




## **`2`** - **Carregamento dos Dados (Data Injection)**

Nesta etapa, carregamos o conteúdo que será usado como base de conhecimento do nosso sistema RAG.




No nosso exemplo, vamos trabalhar com um artigo científico sobre RAGs, em formato PDF. O conteúdo no arquivo PDF será transformado em objetos do tipo `Document`, que são estruturas usadas internamente pela LangChain para organizar o texto e suas informações associadas (como o nome do arquivo ou número da página).


- Criar **um `Document` por página**, quando o conteúdo faz sentido separadamente
- **Concatenar todas as páginas** em um único `Document`, quando o conteúdo depende de contexto global

> A seguir, carregamos o PDF, extraímos o texto de cada página, e juntamos tudo em um único `Document` com metadados.

In [ ]:
# Data Injection com um PDF

loader = PyPDFLoader("./meus_arquivos/RAG_LLM.pdf") # Cria um loader para ler o arquivo. Substitua pelo seu caminho do PDF.
pages = loader.load_and_split() # Carrega o PDF e divide o conteúdo em páginas, onde cada uma vira um Document. Agora, Pages é uma lista de objetos Document (um Document equivale a uma pagina do PDF).

pdf_source = pages[0].metadata['source'] # Da primeira pagina do documento, pega o metadado 'source', que é o caminho do arquivo PDF. Será usado como a fonte do seu documento final concatenado. 

list_document = [] # Cria uma lista vazia para armazenar o documento final concatenado. 

# Junta o conteúdo de todas as páginas em um único texto, separando cada página por seis quebras de linha.
concatenated_text = "\n\n\n\n\n\n".join(
    [page.page_content for page in pages])
# Cria um novo objeto Document com o conteúdo concatenado do PDF inteiro e o metadado 'source' que indica a origem do PDF.
document = Document(
    metadata={'source': pdf_source}, page_content=concatenated_text)
# Adiciona o Document concatenado à lista de documentos.
list_document.append(document)

**`E se forem múltiplos documentos?`**

Para carregar múltiplos arquivos, é necessário implementar um for loop que percorre a lista, aplicando o mesmo processo para cada arquivo dela:

- Cria um loader

- Carrega o PDF e divide o conteúdo em páginas

- Pega o metadado da primeira página do documento

- Junta o conteúdo de todas as páginas em um único texto

- Cria um novo objeto Document com o conteúdo concatenado do PDF inteiro e adiciona à list_document
    
- OBS: essa list_document tem que ser criada fora do loop



In [30]:
# Data Injection com vários PDFs

list_document = [] # Cria uma lista vazia para armazenar o documento final concatenado. 
pasta = "./meus_arquivos" # Define o caminho da pasta onde estão os arquivos PDF.
pdf_path = [os.path.join(pasta, f) for f in os.listdir(pasta) if f.endswith('.pdf')] # Cria uma lista contendo os caminhos de todos os arquivos PDF na pasta especificada.

for i in pdf_path: # Para cada arquivo PDF na lista pdf_path:
    loader = PyPDFLoader(i) # Cria um loader para ler o arquivo
    pages = loader.load_and_split() # Carrega o PDF e divide o conteúdo em páginas, onde cada uma vira um Document. Agora, Pages é uma lista de objetos Document (um Document equivale a uma página do PDF).

    pdf_source = pages[0].metadata['source'] # Da primeira página do documento, pega o metadado 'source', que é o caminho do arquivo PDF. Será usado como a fonte do seu documento final concatenado. 


    # Junta o conteúdo de todas as páginas em um único texto, separando cada página por seis quebras de linha.
    concatenated_text = "\n\n\n\n\n\n".join(
        [page.page_content for page in pages])
    # Cria um novo objeto Document com o conteúdo concatenado do PDF inteiro e o metadado 'source' que indica a origem do PDF.
    document = Document(
        metadata={'source': pdf_source}, page_content=concatenated_text)
    # Adiciona o Document concatenado à lista de documentos.
    list_document.append(document)

#### **`MetaData: O que é, Como Tratamos e Por que precisamos?`**


Como vimos na seção de **Data Injection**, ao carregar documentos (como arquivos PDF), cada página ou trecho de texto é transformado em um objeto `Document`. Esses objetos não carregam apenas o conteúdo textual, eles também trazem consigo um campo muito importante: a `metadata`.

##### **O que é Metadata?**

**Metadata** são **informações adicionais sobre o conteúdo**, como:
- O nome do arquivo de origem ('source')
- O número da página original ('page')
- Qualquer outra informação contextual que você queira adicionar (ex: autor, categoria, data)

Exemplo de **Document** com metadata:
```
Document(
    page_content="Texto da página 1...",
    metadata={
        'source': 'contrato.pdf',
        'page': 0
    }
)
```

##### **Como tratamos no projeto?**
Durante o carregamento e pré-processamento dos documentos:

- Cada página lida com o PyPDFLoader já vem com metadados automaticamente

- Quando concatenamos páginas em um único Document, preservamos o source

- No chunking, os pedaços menores herdam automaticamente os metadados do documento original

**Isso é importante porque permite rastrear a origem da informação utilizada na resposta, contribui para a construção de respostas mais confiáveis e transparentes, possibilita a exibição da fonte (como “Fonte: guia.pdf”) e viabiliza filtros por tipo de documento em consultas mais avançadas.**


`Fontes:`


- Carregamento de Dados de grandes bancos de dados: https://www.crossml.com/build-a-rag-data-ingestion-pipeline/











---



## **`3`** - **Chunking (Dividindo os Documentos em Pedaços Menores)**

Depois de carregar os documentos, o próximo passo é o chunking que consiste em dividir textos longos, como os Documents que acabamos de gerar, em pedaços menores chamados de chunks. 






##### **Essa etapa é fundamental porque:**
- LLMs possuem um limite de tokens por entrada, então o texto longo inteiro não seria suportado pelo modelo.
- Menos é mais: trabalhar com pedaços menores melhora a performance da recuperação de contexto.
- Preserva a coerência e relevância durante a resposta, já que cada pedaço representa uma unidade de informação.

#### **`Como fazemos o chunking?`**

Utilizamos uma ferramenta do Langchain chamada "RecursiveCharacterTextSplitter" que tenta quebrar os textos com base em "separadores hierárquicos", estabelece um tamanho máximo para cada chunk e mantém uma sobreposição entre os pedaços para preservar contexto entre eles. 

#### **`Mas o que são separadores hierárquicos, e como escolhemos eles?`**

São "símbolos de quebra" usados para tentar dividir o texto de forma natural e lógica, sem quebra de palavras. 
No código abaixo, usamos a lista default de separadores:

("\n\n", "\n", " ", "")


##### **O processo funciona assim:**

1. Primeiro tenta dividir por \n\n (duas quebras de linha — parágrafo)

2. Se não der, tenta \n (quebra de linha simples)

3. Depois por " "(fim de frase)

4. E se não conseguir mais dividir logicamente, a ferramenta realiza a quebra forçada ("") de acordo com o tamanho máximo (chunk_size)



##### **Como escolher bons separadores?**

O tipo de separadores que escolhemos varia de acordo com o **tipo de documento que estamos carregando**.
É importante considerar a **estrutura natural do texto**, os **pontos lógicos de separação de ideias**, e o **formato dos dados**.
Algumas instruções no processo de escolha dos seus separadores são: 

- Se o texto estiver dividido em parágrafos visíveis, use \n\n como separador principal.
- Se cada linha for uma unidade (como listas, código ou logs), use \n como separador principal. 
- A sequência de lista default ("\n\n", "\n", " ", "") geralmente funciona melhor para arquivos e PDFs.
- Um bom separador deve dividir o texto em blocos que façam sentido isoladamente, sem cortar frases importantes no meio.
- Use uma lista ordenada de separadores do mais forte (parágrafos) ao mais fraco (palavras).
- Se está em dúvida, comece com a sequência default e vá refinando de acordo com os resultados.



#### **`Tamanho Máximo (chunk_size) com Sobreposição: O que é, e como defini-lo?`**

É o tamanho máximo (em caracteres) que cada chunk pode ter, garantindo que o texto fique curto o suficiente para ser processado, mas longo o bastante para manter o contexto.

##### **Como definir um chunk_size ideal?**

O chunk_size ideal depende de três fatores principais:

- O limite de contexto do modelo de linguagem que você está usando

- A estrutura e densidade do conteúdo dos seus documentos

- O objetivo da aplicação (responder perguntas, resumir, classificar, etc)

O ideal é analisar o seu documento e definir um parâmetro inicial, e, a partir daí testar com diferentes tamanhos no seu conteúdo real, verificar quantos chunks são gerados, qual a qualidade da recuperação e se há cortes abruptos em frases ou parágrafos, e ir ajustando até achar o tamanho ideal para o seu documento, ou para o seu tipo de documento. 



#### **`E a sobreposição (chunk_overlap)?`**

A sobreposição (chunk_overlap) define quantos caracteres do final de um chunk serão repetidos no início do próximo. Essa repetição é importante para preservar a continuidade das ideias, principalmente quando uma informação começa no final de um chunk e continua no seguinte.
Sem essa sobreposição, o modelo pode interpretar os pedaços como blocos desconectados, prejudicando a precisão da recuperação e da geração de respostas.

Por exemplo, se chunk_size = 300 e chunk_overlap = 50, os chunks serão criados mais ou menos assim:

- Chunk 1 → caracteres 0 a 300

- Chunk 2 → caracteres 250 a 550

- Chunk 3 → caracteres 500 a 800

Note que os últimos 50 caracteres do chunk anterior reaparecem no início do próximo, garantindo que o modelo mantenha o fio da narrativa ou explicação.


##### **Como definir um chunk_overlap ideal?**

A escolha ideal do chunk_overlap depende do nível de dependência entre as frases ou trechos do seu documento: quanto mais um trecho depende do anterior para fazer sentido, maior deverá ser o chunk_overlap. 
Analogamente à escolha do chunk_size,  O ideal é analisar o seu documento e definir um parâmetro inicial, e, a partir daí testar com diferentes tamanhos no seu conteúdo real, verificar qual a qualidade da recuperação e se há cortes abruptos em frases ou parágrafos, e ir ajustando até achar o tamanho ideal de sobreposição para o seu tipo de documento.


In [31]:
chunk_size = 200 # Define o tamanho máximo de cada chunk para 200 caracteres.
chunk_overlap = 100 # Define a sobreposição entre os chunks como 100 caracteres.
separators = ["\n\n", "\n", " ", ""] # DEfine os separadores hierárquicos

# Cria um objeto 'text_splitter' que vai gaurdar as regras para cortar os documentos em pedaços menores (chunks), usando a ferramenta do Langchain 'RecursiveCharacterTextSplitter
# Esse divisor é recursivo: tenta usar os separadores em ordem sem ultrapassar o limite do chunk_size.
text_splitter = RecursiveCharacterTextSplitter( # A divisão respeita o chunk_size, o chunk_overlap e os separadores hierárquicos definidos.
    chunk_size=chunk_size, 
    chunk_overlap=chunk_overlap, 
    separators=separators
    )
# As regras da divisão definidas em 'text_splitter' são executadas por 'split.documents' na list_documents, e os pedaços da divisão (chunks) são armazenados na variável 'chunks'.
chunks = text_splitter.split_documents(list_document)

`Fontes:` 

- Guia Definitivo de Chunking: https://www.robertodiasduarte.com.br/guia-definitivo-de-chunking-para-rag-e-llms-estrategias-essenciais/
- Five Levels of Chunking Strategies: https://medium.com/@anuragmishra_27746/five-levels-of-chunking-strategies-in-rag-notes-from-gregs-video-7b735895694d
- Guide for Chunking Phase: https://learn.microsoft.com/en-us/azure/architecture/ai-ml/guide/rag/rag-chunking-phase


---

## **`4`** - **Vector Store - Criação e Armazenamento de Embeddings**

Agora que dividimos os documentos em chunks, o próximo passo é transformar esses pedaços em embeddings (vetores numéricos), e armazená-los em uma estrutura chamada 'Vector Store'.




#### **`O que é uma vector store?`**

Uma Vector Store é um  banco de dados que armazena embeddings (vetores numéricos) junto com seus textos originais. Ela permite que, ao receber uma nova pergunta, possamos transformá-la em vetor e **buscar os chunks mais parecidos** com base em similaridade vetorial.

#### **`Mas primeiro: O que são embeddings, e como os criamos?`**

Embeddings são vetores numéricos que representam textos (palavras, frases ou parágrafos) de forma que **proximidade entre vetores reflita proximidade de significado**. São valores matemáticos, transformados de linguagem humana, após serem processados por algoritmos, 

**`Exemplo:`** os embeddings de “carro” e “automóvel” estarão próximos no espaço vetorial, mesmo com palavras diferentes.

Para gerar os embeddings, usamos um modelo de embedding da OpenAI chamado "text-embedding-3-small", que transforma cada chunk em um vetor denso de alta dimensionalidade. 


##### **Como o modelo de embedding da OpenAI foi criado?**

Existem vários modelos de embeddings, como esse da OpenAI que estamos usando, que são modelos de machine learning, criados por meio de redes neurais treinadas em grandes volumes de texto.

> Vamos entender como esse modelo de Embedding da OpenAI foi criado

O modelo é alimentado com grandes volumes de texto e executa a tarefa de previsão automática da palavra ausente em uma frase.
    Exemplo: "O céu está ___" → o modelo tenta prever "azul".

O processo para realizar essa tarefa de treinamento ocorre da seguinte forma:

1. **Processamento do texto pelas camadas neurais:** O texto é convertido em números (tokens) e passa por várias camadas diferentes da rede neural, onde cada camada extrai relações e padrões mais complexos (como estrutura gramatical, significado, tom e contexto).

2. **Geração de um vetor interno (embedding)**: Antes da camada final, onde a palavra ausente é escolhida, é gerada uma representação vetorial daquele texto, chamada embedding, que representa o significado do texto. 

3. **Cálculo do erro:** Na camada final, onde é feita a previsão, o modelo compara a palavra prevista com a real, e faz o cálculo do erro (função de perda).

4. **Backpropagation e Ajuste dos Pesos:** Com o erro final calculado, entra o processo de backpropagation, em que o modelo vai camada por camada e determina quais conexões (pesos) em cada camada ajudaram ou atrapalharam a chegar na resposta errada ao calcular o gradiente, que o diz basicamente se deve aumentar ou diminuir o peso, e quanto, e vai ajustando os pesos conforme o gradiente. Isso acontece milhares de vezes até o modelo aprender.

5. **Aproximação de significados semelhantes no espaço vetorial:** Após o treinamento, o modelo é capaz de gerar embeddings (vetores) parecidos para textos com significados parecidos.
Por exemplo, frases como "o carro está na garagem" e "o automóvel está estacionado" terão embeddings próximos.

Após o treinamento, o modelo é usado diretamente, como estamos fazendo aqui: envia um texto, e retorna o embedding, um vetor que representa seu significado em um espaço matemático de alta dimensão. 

**`ATENÇÃO`**: A explicação dada acima se refere unicamente ao treinamento do modelo de Embedding da OpenAI que usamos nesse template. Existem outras formas de criar um modelo. 

#### **`E por que usamos Embeddings?`**

Em uma pipeline RAG, queremos que o modelo de linguagem gere respostas com base em **informações relevantes e específicas** dos nossos documentos.  
Para isso, precisamos de uma forma de **buscar os trechos mais parecidos** com a pergunta feita pelo usuário — e é aqui que os **embeddings entram**.

##### **Problemas com buscas tradicionais**

Um sistema de busca baseado em palavras-chave (como Crtl-F)  **só encontra trechos com os mesmos termos exatos**, e não tem suporte para  **sinônimos, variações e perguntas indiretas**, comuns no contexto de perguntas do usuário. Portanto, esse sistema falha porque não entende o significado.


Embeddings resolvem esse problema ao **converter textos em vetores numéricos que capturam seu significado**.  
Com isso, conseguimos:

- Converter a **pergunta do usuário em um vetor**
- Comparar com os **vetores dos chunks armazenados**
- Retornar os **mais semelhantes semanticamente**


##### **Onde os embeddings são usados? No Retriever.**

O **retriever** é o componente da pipeline RAG responsável por:

1. Receber a pergunta do usuário
2. Gerar seu embedding (vetor)
3. Comparar esse vetor com os embeddings dos chunks (na Vector Store)
4. Trazer os chunks mais próximos no espaço vetorial

Ou seja, o retriever **usa embeddings para encontrar o conteúdo mais relevante**, mesmo que a pergunta e o documento usem palavras diferentes.



O retriever precisa **entender o contexto da pergunta**, e não apenas bater palavras exatas.  
Embeddings tornam isso possível, permitindo buscas por **significado**, e não só por termos.  
Isso melhora a precisão, a flexibilidade e a capacidade de resposta da aplicação.

##### **E como comparamos embeddings?**

Para saber **quais embeddings estão mais próximos**, usamos uma métrica chamada **similaridade por cosseno** (*cosine similarity*).

Essa métrica mede o **ângulo entre dois vetores**, e o valor da similaridade varia de:

- `1` → vetores idênticos (máxima similaridade)
- `0` → vetores completamente diferentes
- `-1` → vetores opostos (raro em embeddings de texto)

Essa técnica permite recuperar, de forma matemática e precisa, **os chunks mais semanticamente próximos da pergunta**.

**`ATENÇÃO`**: Há outras formas de comparar os embeddings, mas nesse *template* usamos o método de similaridade por cosseno, ou *cosine similarity*.

In [32]:
embeddingsmodel = OpenAIEmbeddings(model="text-embedding-3-small") # Cria a variável 'embeddingsmodel' que acessa o modelo de embedding da OpenAI por meio da classe OpenAIEmbeddings, que te conecta com a API da OpenAI. 
vectorstore = InMemoryVectorStore(embeddingsmodel) # Cria a variável "vectorstore" que representa um objeto da classe "InMemoryVectorStore", que chama a variável criada acima para usar o modelo de embeddings, gerar vetores a partir dos chunks gerados anteriormente, e armazena-os na memória. 
_ = vectorstore.add_documents(documents=chunks) # Adiciona os chunks de texto a Vector Store, possibilitando buscas por similaridade entre os chunks de texto.

#### **`Teste a busca por similaridade`**

A célula a seguir serve como um teste manual da etapa de vector store e busca por similaridade, não faz parte da pipeline RAG em produção.

É uma forma de validar, antes de criar o retriever e a chain, que a base vetorial está pronta e retornando resultados coerentes.


**`ATENÇÃO`**: É recomendado que as perguntas de teste sejam feitas em inglês, para haver perda mínima de contexto em uma tradução.


In [33]:
# Faz uma busca por similaridade. A pergunta é transformada em um vetor (embedding), e esse vetor é comparado com todos os vetores armazenados na Vector Store (os chunks). Ele calcula a similaridade cosseno entre eles, e retorna os 5 chunks mais semelhantes a pergunta, em termos semânticos.
vectorstore.similarity_search(
    "How does the RAG method combine document retrieval with natural language generation?",
    k=5,
)

# Colocar para salvar o vectorstore localmente para não ficar rodando o modelo de embedding

[Document(id='b8a9b11c-0561-4ef2-8bd6-659a90d1312e', metadata={'source': './meus_arquivos\\RAG_LLM.pdf'}, page_content='overcome challenges, Retrieval-Augmented Generation (RAG)\nenhances LLMs by retrieving relevant document chunks from\nexternal knowledge base through semantic similarity calcu-'),
 Document(id='e64ce71f-3a0e-4f9e-91ef-63ce7f165a29', metadata={'source': './meus_arquivos\\RAG_LLM.pdf'}, page_content='of developing specialized strategies to integrate retrieval with\nlanguage generation models, highlighting the need for further\nresearch and exploration into the robustness of RAG.'),
 Document(id='206aee48-cd0f-4ec8-8b74-b3ac1873815f', metadata={'source': './meus_arquivos\\RAG_LLM.pdf'}, page_content='3\nFig. 2. A representative instance of the RAG process applied to question answering. It mainly consists of 3 steps. 1) Indexing. Documents are split into chunks,'),
 Document(id='46f523eb-37e1-4391-9f38-61fd2f9a022a', metadata={'source': './meus_arquivos\\RAG_LLM.pdf'}, pa

`Fontes:`

- Guia para gerar embeddings: https://learn.microsoft.com/en-us/azure/architecture/ai-ml/guide/rag/rag-generate-embeddings

- Como selecionar um modelo de embedding: https://galileo.ai/blog/mastering-rag-how-to-select-an-embedding-model

- Embeddings e Vetorização: https://joaomarcuraa.medium.com/rag-embeddings-e-vetoriza%C3%A7%C3%A3o-potencializando-a-ia-com-python-e704a39699dd

- Vector Search e Embeddings: https://www.thecloudgirl.dev/blog/the-secret-sauce-of-rag-vector-search-and-embeddings

- Embeddings: https://platform.openai.com/docs/guides/embeddings

- Modelo de Embedding da OpenAI: https://platform.openai.com/docs/guides/embeddings#embedding-models



---

## **`5`** - **Retriever**


Agora que já temos todos os nossos **chunks armazenados na Vector Store com seus embeddings**, o próximo passo é preparar o **componente que será responsável por buscar automaticamente os trechos relevantes de texto** quando o usuário fizer uma pergunta: o **Retriever**.

**`ATENÇÃO`**: O teste acima fazendo essa busca pela ferramenta similarity_search foi **manual**. Agora essa busca será automatizada e feita internamente pelo retrieval.

#### **`Relembrando: O que é Retriever?`**


O **Retriever** é uma camada que se conecta à Vector Store para realizar buscas **semânticas** (por significado), e **retornar os chunks mais relevantes** com base em uma consulta textual.

> Ele é a ponte entre a pergunta do usuário e os pedaços de texto armazenados que mais se aproximam dessa pergunta em termos de conteúdo.

Enquanto a Vector Store **armazena** os embeddings, o Retriever **faz a busca neles**, de forma otimizada.

**`ATENÇÃO`**:Há vários tipos de retrievers (Naive, Parent Document, Self-Query, Contextual Comprehension), mas nesse template estamos implementando o Naive Retriever, o mais simples. 


In [34]:
# Cria uma variavel 'retriever' que acessa a vector store e aplica o método 'as_retriever' para transformar a Vector Store em um objeto (retriever). 
# O parâmetro search_kwargs={"k": 5} define que, a cada consulta, ele deve retornar os 5 chunks mais semelhantes.
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

`Fontes:`

- Técnicas de Retriever: https://towardsdatascience.com/advanced-retriever-techniques-to-improve-your-rags-1fac2b86dd61/


---

## **`6`** - **Geração de Prompt - Montando a pergunta com contexto**

Agora que já temos nosso **retriever configurado**, pronto para buscar automaticamente os **chunks mais relevantes** da base vetorial com base na pergunta do usuário, o próximo passo é **preparar a entrada que será enviada ao modelo de linguagem** (LLM).

Esse passo é fundamental: o **modelo precisa receber tanto o contexto quanto a pergunta**, organizados de forma clara, para gerar uma **resposta precisa e fiel** ao conteúdo.


#### **`Mas afinal, o que é prompt?`**

Um prompt é o texto de entrada que você envia para um modelo de linguagem (LLM), como o da OpenAI.

Ele serve para instruir o modelo sobre o que ele deve fazer e qual informação ele deve usar para gerar uma resposta.

O modelo de linguagem não “sabe” o que fazer sozinho — ele só reage ao que está no prompt.
Se o prompt for mal formulado, ele pode:

- Ignorar o contexto

- Inventar respostas

- Responder de forma genérica

Um bom prompt garante que o modelo use o contexto corretamente e responda com precisão.


#### **`E o que vamos fazer com ele agora?`**

Vamos criar um **prompt estruturado**, ou seja, um *template* que:

prompt completo:

Seu prompt é composto por:

- Mensagem de sistema → define o comportamento do modelo (que não deve ser apresentado ao usuário final)

- Mensagem do usuário → traz a pergunta

- Contexto recuperado → alimentado dentro do template

Primeiro importamos a ferramenta do Langchain 'ChatOpenAI' que permite usar os modelos de chat da OpenAI dentro do Langchain, se conectando à API da OpenAI, enviando o prompt, e recebendo a resposta. 



##### **Importando ChatOpenAI**

In [35]:

# Importa a ferramenta que permite usar os modelos de chat da OpenAI dentro do Langchain (conectar a API, envia um prompt e recebe a resposta)
from langchain_openai import ChatOpenAI


##### **Criando o Modelo de Prompt**

Como vimos acima, o modelo de prompt é composto pela mensagem de sistema, pela mensagem do usuário, e pelo contexto recuperado (do retriever). 

**`ATENÇÂO`**: A mensagem de sistema normalmente é adaptada de acordo com o documento e com o objetivo da aplicação, e não deve ser apresentada ao usuário final. 

**Etapas:**

1. Primeiro, vamos criar a 'system_prompt' para guardar a mensagem de sistema e o contexto recuperado. 

2. Depois, vamos juntar esse 'system_prompt' com a entrada do usuário e guardá-lo em 'prompt'. Faremos isso pela ferramenta 'ChatPromptTemplate', que serve para montar os prompts no formato de mensagens, permitindo criar um template de prompt com partes variáveis.

3. Seguindo o formato de mensagem, definimos 'system' como 'system_prompt', representando o papel e o comportamento da IA. E definimos 'human' como a pergunta feita pelo usuário.

In [36]:
# Criando uma string longa chamada "system_prompt" que será usada como mensagem do sistema, uma das partes do prompt, que define o comportamento e papel do modelo durante a resposta.
system_prompt = (
    "You are an assistant for question-answering tasks for compliance documents. " # Define o papel da IA
    "Use the following pieces of retrieved context to answer " # Diz de onde a IA deve buscar as informações para responder, além de seu conhecimento interno
    "the question. If you don't know the answer, say that you " # Uma instrução de segurança para impedir que a IA invente respostas
    "don't know. Keep the answer as close as retrieved context as possible." # Outra instrução de segurança para impedir que a IA invente respostas
    "\n\n"
    "Context: {context}" # Espaço para inserir o contexto recuperado, que são os chunks mais relevantes para a pergunta feita, obtido pelo retriever.
)

# Juntando o system_prompt (contendo a mensagem do sistema e o contexto) com a pergunta do usuário pelo ChatPromptTemplate, ferramenta  que permite criar prompts estruturados para modelos.
prompt = ChatPromptTemplate.from_messages( # o 'from_messages' é um método dessa ferramenta que cria um template de prompt a partir de uma lista de mensagens.
    [
        ("system", system_prompt), # A primeira mensagem é do tipo "system", que define o papel da IA e o contexto que ela deve usar para responder.
        ("human", "Question: {input}"), # A segunda mensagem é do tipo "human", que representa a pergunta feita pelo usuário. O "{input}" será substituído pela pergunta real.
    ]
)

##### **Teste da construção final do prompt**

Nessa etapa vamos testar se o prompt foi construído corretamente antes de enviá-lo para o modelo de linguagem (LLM). Isso inclui verificar se:
- O contexto recuperado está no lugar certo.
- A pergunta do usuário está no lugar certo.
- O formato da mensagem está seguindo a estrutura esperada. 



In [37]:
# Chamamos o prompt com o método 'invoke', passando um dicionário com as chaves "context" e "input".
# A chave "context" recebe o contexto recuperado pelo retriever, e a chave "input" recebe a pergunta do usuário, e as duas são salvas em 'message_list'.
message_list = prompt.invoke({
    "context": "overcome challenges, Retrieval-Augmented Generation (RAG)\n enhances LLMs by retrieving relevant document chunks from\nexternal knowledge base through semantic similarity calculation.",
    "input": "How does the RAG method combine document retrieval with natural language generation?"
})

# Itera sobre a lista de mensagens retornadas pelo prompt e chama o método 'pretty_print' para exibir cada mensagem de forma legível.
for msg in message_list.messages:
    msg.pretty_print()

================================ System Message ================================

You are an assistant for question-answering tasks for compliance documents. Use the following pieces of retrieved context to answer the question. If you don't know the answer, say that you don't know. Keep the answer as close as retrieved context as possible.

Context: overcome challenges, Retrieval-Augmented Generation (RAG)
 enhances LLMs by retrieving relevant document chunks from
external knowledge base through semantic similarity calculation.
================================ Human Message =================================

Question: How does the RAG method combine document retrieval with natural language generation?


`Fontes:`

- Engenharia de Prompt em RAG: https://ventiladigital.com.br/blog/estrategias-eficazes-para-engenharia-de-prompts-em-rag/

- Guia de Prompt - Tecnicas para RAG: https://www.promptingguide.ai/pt/techniques/rag



---

## **`7`** - **RAG Chain**

Chegamos à etapa final da pipeline: a RAG Chain. Esse é o componente que integra todas as partes da pipeline:
- Retriever: busca os chunks mais relevantes para o contexto
- Prompt: organiza o contexto, a mensagem de sistema e a pergunta do usuário em um só. 
- LLM: gera a resposta final

Aqui é onde o sistema pega a pergunta do usuário, recupera o contexto, monta o prompt, e gera a resposta de forma totalmente automatizada. 





O processo vai funcionar da seguinte forma:

1. Primeiro vamos **escolher o modelo e definir alguns parametros**.
2. Depois vamos **conectar os componentes da pipeline, e obter a resposta final da LLM**. Podemos fazer isso de dois modos diferentes, dependendo do nível de personalização necessário na sua aplicação:
 - **LCEL**: uma forma manual e flexível de montar a pipeline. Você conecta componentes usando o operador |, como blocos funcionais encadeados. Ideal para quem precisa de **mais controle**, quer modificar partes da lógica, integrar validações ou adicionar etapas intermediárias.
 - **Create Stuff**: usando funções utilitárias como 'create_stuff_documents_chain' e 'create_retrieval_chain(retriever, doc_chain)', que já **montam e conectam os blocos principais automaticamente**, o que reduz código e acelera o desenvolvimento, sem muita customização e controle. 

**`E quando devo usar LCEL, e quando devo usar Create Stuff?`**

| Caso de uso                                      | Recomendação     |
|--------------------------------------------------|--------------------|
| Preciso de controle sobre cada etapa da pipeline | **LCEL**          |
| Quero alterar o parser, prompt ou fluxo de dados | **LCEL**         |
| Estou fazendo um protótipo funcional             | **Create Stuff**  |
| O projeto é simples e precisa de pouco código    | **Create Stuff**  |
| Quero lógica personalizada (ex: filtros, logs)   | **LCEL**          |
| Quero escalar com menos esforço manual           | **Create Stuff**  |



--





> Agora vamos colocar em prática


##### **1- Escolher o Modelo e Definir os Parâmetros**


In [38]:
# Crio a variável 'llm' que acessa o modelo de chat da OpenAI, usando a classe 'ChatOpenAI' do Langchain, que conecta com a API da OpenAI.
llm = ChatOpenAI(model_name = "gpt-4.1-mini", temperature=0) # Defino o modelo da OpenAI a ser usado, e a temperatura, que controla o quão criativo o modelo deve ser. 



**`O que é temperatura de um modelo?`**

Temperatura é um parâmetro que controla o grau de aleatoriedade ou criatividade nas respostas geradas por modelos como o GPT. 
Essa temperatura varia de 0 a 1, sendo:
- `0` respostas mais diretas, exatas e consistentes.
- `1` respostas mais criativas, abertas e variáveis.

O valor default da temperatura ao usar o ChatOpenAI é 0.7, que significa que o modelo não vai gerar respostas completamente aleatórias, mas também não serão totalmente fixas. 


**`ATENÇÃO`**: A temperatura deverá ser ajustada de acordo com o tipo de documento que foi carregado e o objetivo da aplicação. É recomendado que a temperatura para aplicações de RAG seja 0. 

##### **2.1- LCEL: Langchain Expression Language**

LCEL é uma linguagem declarativa e funcional criada dentro do Langchain, que significa que:
- Declara o que quer que aconteça como: entrada -> prompt -> modelo -> saída 
- Segue princípios da programação funcional:
    - Cada componente é uma função
    - Encadea as funções com invoke()
    - As funções não mudam o estado interno das outras, mas se compõe para formar funções maiores a partir de pequenas. 

Isso permite que você conecte os componentes de forma encadeada, reduzindo código repetitivo e tornando a pipeline reutilizável. 

##### **`Mas e na prática, como funciona?`**

Na prática, o LCEL permite que você monte sua pipeline RAG como um **fluxo contínuo de execução**, conectando os blocos com o operador `|`.  
Cada bloco é um componente que **transforma ou utiliza os dados** antes de passá-los para o próximo.

1. Definir o template do prompt, chamando o retriever e a entrada do usuário
2. Substituímos os placeholders no prompt com os valores reais, e gero o prompt completo
3. Passo o prompt completo para o LLM, que gera uma resposta bruta.
4. Converto a resposta bruta do LLM em uma string limpa pronta para ser exibida. 


In [39]:

# Cria uma variável 'parser' que inicializa a classe 'StrOutputParser', que é usada para converter a resposta gerada pelo modelo em uma string limpa, removendo formatações desnecessárias e deixando o texto pronto para uso.
parser = StrOutputParser()

# Cria uma cadeia RAG completa, que conecta todos os componentes (retriever, prompt, llm e parser) para formar um fluxo funcional usando LCEL
rag_chain = (
    { # Define o formato de entrada esperado pela cadeia
        "context": retriever, # Chama o retriever para buscar o contexto relevante
        "input": RunnablePassthrough() # Usa o RunnablePassthrough para passar a entrada do usuário sem alterações
    }
    | prompt # O dicionário {input, context} é passado para o ChatpromptTemplate, substitui os placeholders {input} e {context} no prompt com os valores reais e gera o prompt completo para a LLM.
    | llm # O LLM é chamado, recebe o prompt completo e gera uma resposta bruta em um formato não estruturado.
    | parser # O parser pega a resposta bruta do modelo, e a converte em uma string limpa e estruturada, pronta para ser usada ou exibida.
    )

 **`Teste Funcional da Pipeline Completa`**

Agora com todos os componentes conectados, é hora de simular o comportamento real da aplicação com uma pergunta de exemplo. 
Para isso, a cadeia é executada pelo método 'invoke' ao passar a pergunta "What are the key issues related to ethics in Brazil?" como string. A variável 'response' contém o resultado da cadeia, que é a resposta da LLM. 
A pergunta é relacionada ao documento carregado e deve ser adaptada conforme o documento.

In [40]:

response = rag_chain.invoke("How does the RAG method combine document retrieval with natural language generation?") # Chama a cadeia RAG com a pergunta do usuário
print(response) # exibir a resposta final gerada pela cadeia RAG, que é uma string limpa e estruturada, pronta para ser usada ou exibida.

The Retrieval-Augmented Generation (RAG) method combines document retrieval with natural language generation by first retrieving relevant document chunks from an external knowledge base through semantic similarity calculations. This retrieval process is integrated with language generation models to enhance responses. Specifically, the RAG process involves indexing documents by splitting them into chunks, retrieving the most relevant chunks based on the input query, and then using these retrieved documents to augment the generation of natural language responses. This synergy between retrieval and generation improves the overall efficiency and quality of the system's outputs.


##### **2.2 - Create Stuff: Funções Utilitárias**

Por mais que já tenhamos gerado a resposta final da LLM em um formato estruturado no passo anterior, o método LCEL é uma forma manual e controlada de montar o fluxo da pipeline, em que você define cada etapa e pode modificar seu fluxo a gosto, ideal para customização. 

Porém, há outro jeito de conectar os componentes e gerar esse fluxo de um modo mais simplificado e automático, sem muita customização e controle, usando funções utilitárias como 'create_stuff_documents_chain' e 'create_retrieval_chain'. Vamos nos aprofundar nesse método agora:

**`1. Primeiro usamos a função do langchain 'create_stuff_documents_chain' que cria uma "document chain", ou seja, uma cadeia responsável por gerar a resposta final da LLM com base em documentos de contexto. Internamente, a função:`**

- Espera receber documentos (normalmente vindos do retriever)
- Insere esses documentos no campo `{context}` do prompt
- Insere a pergunta do usuário no campo `{input}`
- Envia o prompt completo para o modelo de linguagem (LLM)
- Retorna apenas a resposta final gerada pela LLM

Pensa nela como a parte que monta o prompt final e gera a resposta.

**`2. Depois usamos a função 'create_retrieval_chain' que conecta o retriever à cadeia de geração de resposta (criada acima), formando o fluxo completo da RAG. Internamente, a função:`**

- Usa o retriever para buscar documentos relevantes com base na pergunta
- Envia esses documentos para a `question_answer_chain`
- Usa a LLM para gerar a resposta final
- Retorna um dicionário contendo:
  - "answer" → a resposta gerada pela LLM
  - "context" → os documentos utilizados como suporte

Pensa nela como a estrutura completa da RAG, que junta:
- a busca por contexto (retriever)  
- com a geração da resposta (LLM)






In [41]:

# Cria uma cadeia de documentos do tipo "stuff" (junção simples de documentos) usando o modelo de linguagem (LLM) e o prompt definido. Essa cadeia é responsável por gerar a resposta final da RAG com base no contexto recuperado.
question_answer_chain = create_stuff_documents_chain(llm, prompt)

#Cria a cadeia completa da RAG (Retrieval-Augmented Generation), conectando o retriever com a cadeia de geração de resposta (question_answer_chain) que foi definida acima.
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


 **`Teste Funcional da Pipeline Completa`**

Agora com todos os componentes conectados, é hora de simular o comportamento real da aplicação com uma pergunta de exemplo. 
Para isso, a cadeia é executada pelo método 'invoke' ao passar a pergunta "What are the key issues related to ethics in Brazil?" como string. A variável 'res' contém o resultado da cadeia, que é a resposta da LLM. 
A pergunta é relacionada ao documento carregado e deve ser adaptada conforme o documento carregado.

In [42]:
res = rag_chain.invoke({"input": "How does the RAG method combine document retrieval with natural language generation?"}) # Chama a cadeia RAG com a pergunta do usuário
res["answer"] # exibir a resposta final gerada pela cadeia RAG, que é uma string limpa e estruturada, pronta para ser usada ou exibida.

'The Retrieval-Augmented Generation (RAG) method combines document retrieval with natural language generation by first retrieving relevant document chunks from an external knowledge base through semantic similarity calculations. This retrieval step provides pertinent information that is then integrated into the language generation process. By combining "Retrieval," "Generation," and "Augmentation," RAG enhances language models to produce more informed and accurate responses. The process typically involves indexing documents by splitting them into chunks, retrieving relevant chunks based on the query, and then generating responses that incorporate the retrieved information, thereby improving the overall efficiency and robustness of the system.'

 **`Vizualizando o Contexto`**

Após executar a cadeia RAG e obter a resposta, também é possível inspecionar os documentos que foram recuperados como contexto e utilizados como base para a resposta final.


In [43]:
# Percorre cada documento presente na lista de contexto retornada pela RAG
for context in res["context"]:
    print(context.page_content) # Imprime o conteúdo textual (texto bruto) de cada documento recuperado

overcome challenges, Retrieval-Augmented Generation (RAG)
enhances LLMs by retrieving relevant document chunks from
external knowledge base through semantic similarity calcu-
of developing specialized strategies to integrate retrieval with
language generation models, highlighting the need for further
research and exploration into the robustness of RAG.
3
Fig. 2. A representative instance of the RAG process applied to question answering. It mainly consists of 3 steps. 1) Indexing. Documents are split into chunks,
to the RAG process, specifically focusing on the aspects
of “Retrieval”, “Generation” and “Augmentation”, and
delve into their synergies, elucidating how these com-
responses, thus improving the overall efficiency of the RAG
system. To capture the logical relationship between document
content and structure, KGP [91] proposed a method of building


`Fontes:`

- Construindo a RAG Chain com LCEL: https://towardsdatascience.com/building-a-rag-chain-using-langchain-expression-language-lcel-3688260cad05/
- Create Stuff Documentation: https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html

---

## **`Conclusão`**

Com esta pipeline completa de RAG (Retrieval-Augmented Generation) usando Langchain e OpenAI, conseguimos:






- Carregar e preparar documentos em PDF como base de conhecimento.

- Dividir o conteúdo em chunks otimizados para recuperação.

- Gerar embeddings vetoriais e armazená-los em uma vector store.

- Configurar um retriever para buscas semânticas.

- Montar prompts estruturados e personalizados.

- Criar fluxos de execução automatizados com LCEL e funções utilitárias.

- Simular e testar o comportamento real da aplicação de forma modular e reutilizável.

Esse fluxo é uma base robusta para qualquer aplicação que envolva respostas baseadas em documentos, como assistentes jurídicos, analisadores de contratos, suporte técnico inteligente, entre outros.


**`Referências e Materiais Recomendados`**


**Documentação Oficial**

LangChain Documentation – documentação completa, com tutoriais e exemplos práticos. (https://python.langchain.com/docs/introduction/)

LangChain Prompt Guide – guia oficial para entender e construir bons prompts.(https://js.langchain.com/docs/how_to/graph_prompting/)

OpenAI API Reference – referência da API usada para gerar embeddings e respostas. (https://platform.openai.com/docs/overview)

**Artigos e Guias** 

LangChain: Getting Started (Towards Data Science) – uma introdução prática e bem explicada. (https://www.datacamp.com/tutorial/introduction-to-langchain-for-data-engineering-and-data-applications)

Advanced Retriever Techniques – para explorar outros tipos de retrievers além do Naive Retriever. (https://towardsdatascience.com/advanced-retriever-techniques-to-improve-your-rags-1fac2b86dd61/)

Vector Databases 101 (Pinecone) – excelente explicação sobre o papel das vector stores em sistemas de IA. (https://www.pinecone.io/learn/vector-database/)

**Extras para se aprofundar**

LangChain YouTube Channel – vídeos curtos e objetivos para entender o funcionamento das ferramentas. (https://www.youtube.com/@LangChain)

Playlist Completa sobre RAG do Langchain. (https://www.youtube.com/playlist?list=PLfaIDFEXuae2LXbO1_PKyVJiQ23ZztA0x)